# Podstawy programowania w analizie danych

## Tomasz Rodak

2017/2018, semestr letni

Wykład XIII

# Wyrażenia regularne, cz. 2

In [2]:
import re

## Zachłanny vs leniwy

**Zagadka.** Do czego dopasuje sie wyrażenie
```
\d+
```
w łańcuchu
```
123456789
```

Do całego łańcucha, gdyż kwantyfikatory są domyślnie **zachłanne** (*greedy*) -- spośród możliwych dopasowań wybierane jest to o maksymalnej długości. 

In [6]:
re.search(r'\d+', '123456789').group()

'123456789'

Znak zapytania `?` zmienia zachłanny mechanizm kwantyfikatora na **leniwy** (*lazy*) -- spośród możliwych dopasowań wybierane jest najkrótsze.

In [8]:
re.search(r'\d+?', '123456789').group()

'1'

### Zagadka

Jaki tekst zobaczymy tutaj?

```python
re.search(r'\d*?', '123456789').group()
```

Pusty łańcuch -- gwiazdka to zero lub więcej wystąpień.

In [10]:
re.search(r'\d*?', '123456789').group()

''

Jednak mechanizm domyślny jest zachłanny.

In [11]:
re.search(r'\d*', '123456789').group()

'123456789'

### Uwaga

Dopasowanie puste to nie to samo co brak dopasowania!

Tutaj dopasowania nie ma -- dostajemy `None`.

In [16]:
brak = re.search('\d', 'abc')

print(brak)

None


Ale tutaj jest. Dopasowany łańcuch jest pusty.

In [13]:
puste = re.search('\d?', 'abc')

bool(puste), puste.group()

(True, '')

## Alternatywa

Znak pionowej kreski 
```
regex1|regex2|...
```
tworzy alternatywę wyrażeń regularnych.

In [4]:
pies_lub_kot = re.compile(r'pies|kot')

list(pies_czy_kot.finditer('pies czyli kot'))

[<_sre.SRE_Match object; span=(0, 4), match='pies'>,
 <_sre.SRE_Match object; span=(11, 14), match='kot'>]

Zwracany jest pierwszy dopasowany składnik, dlatego kolejność może mieć znaczenie.

Na przykład

In [5]:
funkcja = re.compile(r'set|setValue')

funkcja.search('def setValue():')

<_sre.SRE_Match object; span=(4, 7), match='set'>

ale

In [6]:
funkcja = re.compile(r'setValue|set')

funkcja.search('def setValue():')

<_sre.SRE_Match object; span=(4, 12), match='setValue'>

## Grupowanie

Do grupowania wyrażeń regularnych służą nawiasy okrągłe `()`. Fragmenty wyrażenia ograniczone nawiasami nazywamy **grupami**.

Kwantyfikator występujący bezpośrednio po zamykającym nawiasie okrągłym odnosi się do wszystkiego co ten nawias ogranicza.

In [30]:
import random

tekst = ''.join(random.choice('abc') for _ in range(100))
tekst

'ccabcbbbacbccabcccbcaabcabcabcaabaaabbbbbacbacacacbbbbcaccaaaabccacbbaccacabccbccbacaabccbbbacbbcbbb'

In [31]:
list(re.finditer(r'(abc)+', tekst))

[<_sre.SRE_Match object; span=(2, 5), match='abc'>,
 <_sre.SRE_Match object; span=(13, 16), match='abc'>,
 <_sre.SRE_Match object; span=(21, 30), match='abcabcabc'>,
 <_sre.SRE_Match object; span=(61, 64), match='abc'>,
 <_sre.SRE_Match object; span=(74, 77), match='abc'>,
 <_sre.SRE_Match object; span=(85, 88), match='abc'>]

### Zagadka

Do czego dopasowuje się wyrażenie `rozum`?

In [35]:
rozum = re.compile(r'rozum(u|owi|em|ie|y|ów|om|ami|ach)?')

Do rzeczownika *rozum* z uwzględnieniem odmiany przez przypadki.

Sprawdzimy to na podstawie fragmentu z *Krytyki czystego rozumu* I. Kanta.
```
Z tego wszystkiego wypływa tedy idea odrębnej umiejętności,
mogącej się nazywać *Krytyką czystego rozumu*. 
Bo rozum jest władzą, dostarczającą nam *zasad* poznania a priori.
A więc czystym rozumem jest ten, co zawiera 
zasady poznania czegoś zgoła a priori.
```

In [36]:
s = '''Z tego wszystkiego wypływa tedy idea odrębnej umiejętności,
mogącej się nazywać *Krytyką czystego rozumu*. 
Bo rozum jest władzą, dostarczającą nam *zasad* poznania a priori.
A więc czystym rozumem jest ten, co zawiera zasady poznania czegoś zgoła a priori.'''

In [37]:
list(rozum.finditer(s))

[<_sre.SRE_Match object; span=(98, 104), match='rozumu'>,
 <_sre.SRE_Match object; span=(111, 116), match='rozum'>,
 <_sre.SRE_Match object; span=(190, 197), match='rozumem'>]

### Zagadka

A do czego dopasuje się to wyrażenie?
```
\d{1,3}(,\d{3})+
```

Wypróbujemy je na tabelce o krajach północnoafrykańskich.

```
Algeria 2,381,740 34,178,188 2009 14 Algiers
Canary Islands (Spain) 7,492 2,154,905 2017 226 Las Palmas de Gran Canaria
Ceuta (Spain) 20 85,107 2017 3,575 --
Egypt 1,001,450 82,868,000 2012 83 Cairo
Libya 1,759,540 6,310,434 2009 4 Tripoli
Madeira (Portugal) 797 245,000 2001 307 Funchal
Melilla (Spain)[139] 12 85,116 2017 5,534 --
Morocco 446,550 34,859,364 2009 78 Rabat
Sudan 1,861,484 30,894,000 2008 17 Khartoum
Tunisia 163,610 10,486,339 2009 64 Tunis
Western Sahara 266,000 405,210 2009 2 El Aaiún
```

In [40]:
liczba = re.compile(r'\d{1,3}(,\d{3})+')

s = '''Algeria 2,381,740 34,178,188 2009 14 Algiers
Canary Islands (Spain) 7,492 2,154,905 2017 226 Las Palmas de Gran Canaria
Ceuta (Spain) 20 85,107 2017 3,575 --
Egypt 1,001,450 82,868,000 2012 83 Cairo
Libya 1,759,540 6,310,434 2009 4 Tripoli
Madeira (Portugal) 797 245,000 2001 307 Funchal
Melilla (Spain)[139] 12 85,116 2017 5,534 --
Morocco 446,550 34,859,364 2009 78 Rabat
Sudan 1,861,484 30,894,000 2008 17 Khartoum
Tunisia 163,610 10,486,339 2009 64 Tunis
Western Sahara 266,000 405,210 2009 2 El Aaiún'''

In [41]:
list(liczba.finditer(s))

[<_sre.SRE_Match object; span=(8, 17), match='2,381,740'>,
 <_sre.SRE_Match object; span=(18, 28), match='34,178,188'>,
 <_sre.SRE_Match object; span=(68, 73), match='7,492'>,
 <_sre.SRE_Match object; span=(74, 83), match='2,154,905'>,
 <_sre.SRE_Match object; span=(137, 143), match='85,107'>,
 <_sre.SRE_Match object; span=(149, 154), match='3,575'>,
 <_sre.SRE_Match object; span=(164, 173), match='1,001,450'>,
 <_sre.SRE_Match object; span=(174, 184), match='82,868,000'>,
 <_sre.SRE_Match object; span=(205, 214), match='1,759,540'>,
 <_sre.SRE_Match object; span=(215, 224), match='6,310,434'>,
 <_sre.SRE_Match object; span=(263, 270), match='245,000'>,
 <_sre.SRE_Match object; span=(312, 318), match='85,116'>,
 <_sre.SRE_Match object; span=(324, 329), match='5,534'>,
 <_sre.SRE_Match object; span=(341, 348), match='446,550'>,
 <_sre.SRE_Match object; span=(349, 359), match='34,859,364'>,
 <_sre.SRE_Match object; span=(380, 389), match='1,861,484'>,
 <_sre.SRE_Match object; span=(390, 

## Przechwytywanie grup

* Dopasowania do fragmentów wyrażenia regularnego leżących w nawiasach `()`, czyli do grup, są zapamiętywane.
* Grupy są numerowane **od jedynki** zgodnie z kolejnymi otwierającymi nawiasami. Za grupę zerową można uważać całe wyrażenie regularne.
* Obiekt dopasowania posiada metodę `groups()`, która zwraca krotkę z łańcuchami dopasowanymi do kolejnych grup od numery `1` począwszy.
* Metoda `group()`:
  * wywołana bez argumentu zwraca łańcuch dopasowany do grupy zerowej, czyli do całego wyrażenia.
  * wywołana z argumentem całkowitym `k` zwraca łańcuch dopasowany do `k`-tej grupy.

Wyrażenie `liczba` dopasowuje się do liczb zmiennoprzecinkowych o niepustej części przed i za kropką.

In [95]:
liczba = re.compile(r'-?(\d+)\.(\d+)')

list(liczba.finditer('2 3 1.2 -12.7 0.6'))

[<_sre.SRE_Match object; span=(4, 7), match='1.2'>,
 <_sre.SRE_Match object; span=(8, 13), match='-12.7'>,
 <_sre.SRE_Match object; span=(14, 17), match='0.6'>]

Wyrażenie ma dwie grupy: cechę i mantysę.

In [96]:
liczba.search('-34.123').groups()

('34', '123')

## Kotwice (*anchors*)

Kotwica *zakotwicza* dopasowanie do określonej pozycji. Jest to dopasowanie o zerowej długości.

* `^` -- domyślnie początek łańcucha; w trybie `MULTILINE` początek każdej linii.

* `$` -- domyślnie koniec łańcucha; w trybie `MULTILINE` koniec każdej linii.

* `\b` -- początek lub koniec słowa; dokładniej -- miejsce, które z jednej strony ma znak alfanumeryczny, a z drugiej znak niealfanumeryczny lub koniec łańcucha.

### `\b`

Szukamy słowa `kot` w zdaniu
```
kot wszedł do kotła, gdyż szukał maskotki.
```

Pierwsze podejście

In [98]:
zdanie = 'kot wszedł do kotła, gdyż szukał maskotki.'

list(re.finditer(r'kot', zdanie))

[<_sre.SRE_Match object; span=(0, 3), match='kot'>,
 <_sre.SRE_Match object; span=(14, 17), match='kot'>,
 <_sre.SRE_Match object; span=(36, 39), match='kot'>]

daje nam aż trzy koty.

Dopiero wyrażenie
```
\bkot\b
```
odrzuca dopasowania w słowach zawierających `kot` jako fragment.

In [99]:
zdanie = 'kot wszedł do kotła, gdyż szukał maskotki.'

list(re.finditer(r'\bkot\b', zdanie))

[<_sre.SRE_Match object; span=(0, 3), match='kot'>]

### `$`

Wyrażeniem
```
[^\\]+$
```

dopasujemy nazwy plików ze ścieżek.

```
\User\notatki.txt
\Program files\costam.exe
wazne.tex
C:\User\
C:\temp\folder\jakis.dll
```

Kotwicę chcemy postawić na końcu każdej linii, stosujemy więc tryb `re.MULTILINE`.

In [ ]:
plik = re.compile(r'[^\\]+$', flags=re.MULTILINE)

In [13]:
ścieżki = r'''\User\notatki.txt
\Program files\costam.exe
wazne.tex
C:\User\
C:\temp\folder\jakis.dll'''

In [14]:
for nazwa in plik.finditer(ścieżki):
    print(nazwa.group())

notatki.txt
costam.exe
wazne.tex
jakis.dll


### `^`

Wyrażeniem
```
^\w+
```
pobierzemy nazwy plików bez rozszerzeń z łańcucha
```
prog.py
cos.dll
wykonalny.exe
spis.txt
paczka.tar.gz
```

In [5]:
pliki = '''prog.py
cos.dll
wykonalny.exe
spis.txt
paczka.tar.gz'''

list(re.finditer(r'^\w+', pliki, flags=re.MULTILINE))

[<_sre.SRE_Match object; span=(0, 4), match='prog'>,
 <_sre.SRE_Match object; span=(8, 11), match='cos'>,
 <_sre.SRE_Match object; span=(16, 25), match='wykonalny'>,
 <_sre.SRE_Match object; span=(30, 34), match='spis'>,
 <_sre.SRE_Match object; span=(39, 45), match='paczka'>]

## Opcje (flagi)

* `re.IGNORECASE` -- litery małe i duże przestają być rozróżniane.

* `re.MULTILINE` -- znaki `^` i `$` odnoszą się do początku i końca linii, a nie łańcucha.

### Kropka

Znak kropki `.` dopasowuje się domyślnie do każdego znaku z wyjątkiem znak nowej linii `'\n'`.

Użycie flagi `re.DOTALL` powoduje, że kropka dopasowuje się do każdego znaku, łącznie ze znakiem `'\n'`.

## Pełne dopasowanie

Jeśli piszesz program pobierający jakieś dane (np. z klawiatury) od użytkownika, to możesz spróbować wymusić poprawność formatu wyrażeniem regularnym.

Załóżmy, że Twój program pobiera datę w formacie `DD.MM.YYYY`. Wówczas wyrażenie
```
\d\d\.\d\d\.\d\d\d\d
```
odrzuci (część) nieprawidłowych wpisów.

Tu jest dobrze

In [25]:
re.search(r'\d\d\.\d\d.\d\d\d\d', '29.05.2018')

<_sre.SRE_Match object; span=(0, 10), match='29.05.2018'>

ale tu błędnie

In [24]:
re.search(r'\d\d\.\d\d.\d\d\d\d', '293.05.2018')

<_sre.SRE_Match object; span=(1, 11), match='93.05.2018'>

Po tej poprawce
```
^\d\d\.\d\d\.\d\d\d\d$
```
mamy gwarancję, że przekazany dalej łańcuch **cały** dopasowuje się do wyrażenia `\d\d\.\d\d\.\d\d\d\d`

In [27]:
re.search(r'^\d\d\.\d\d.\d\d\d\d$', '293.05.2018')

Wyrażenie
```
^\s*\d\d\.\d\d.\d\d\d\d\s*$
```
sprawi, że test poprawności formatu przejdą łańcuchy z biały znakami na początku i końcu napisu. 

In [31]:
re.search(r'^\s*\d\d\.\d\d.\d\d\d\d\s*$', '   93.05.2018  \n ')

<_sre.SRE_Match object; span=(0, 17), match='   93.05.2018  \n '>

Jak teraz wydobyć datę z łańcucha? Nie wiemy ile białych znaków go poprzedza.

Wykorzystując grupowanie
```
^\s*(\d\d)\.(\d\d).(\d\d\d\d)\s*$
```

Grupy o numerach 1, 2, 3 to odpowiednio dzień, miesiąc, rok.

In [35]:
while True:
    data = input('Data: ')
    dopasowanie = re.search(r'^\s*(\d\d)\.(\d\d).(\d\d\d\d)\s*$', data)
    
    if dopasowanie is not None:
        dzień, miesiąc, rok = dopasowanie.groups()
        print(dzień, miesiąc, rok)
        break
    
    print('Nieprawidłowy format.')

Data: 12.13.12345
Nieprawidłowy format.
Data:     12.13.1234   
12 13 1234


To czego nie sprawdzamy, to zakresy dni i miesięcy. Możesz to zrobić w programie, albo dalej rozbudowywać wyrażenie regularne.

In [36]:
re.search(r'^\s*(\d\d)\.(\d\d).(\d\d\d\d)\s*$', '93.05.2018')

<_sre.SRE_Match object; span=(0, 10), match='93.05.2018'>

## Przykład: znaczniki HTML

Chcemy utworzyć wyrażenie regularne, które dopasuje znaczniki HTML. 

Na przykład, w łańcuchu 

```html
<p>Do not forget to buy <mark>milk</mark> today.</p>
```

należy dopasować `p`, `mark`, `/mark`, `/p`.

### Pierwszy sposób

Zwróć uwagę na leniwy kwantyfikator. Można zastąpić go zachłannym?

In [46]:
tekst = '<p>Do not forget to buy <mark>milk</mark> today.</p>'

html_tag = re.compile(r'<(.+?)>')

for znacznik in html_tag.finditer(tekst):
    print(znacznik.group(1))

p
mark
/mark
/p


### Drugi sposób

Wykorzystujemy negację.

In [47]:
tekst = '<p>Do not forget to buy <mark>milk</mark> today.</p>'

html_tag = re.compile(r'<([^>]+)>')

for znacznik in html_tag.finditer(tekst):
    print(znacznik.group(1))

p
mark
/mark
/p


## `re.sub(wzorzec, zamiennik, łańcuch, count=0, flags=0)`

W tekście `łańcuch` wartość `zamiennik` wstawiana jest w dopasowania wyznaczone przez `wzorzec`.

Poniżej znajduje się raczej nieczytelna lista dziesięciu punktów płaszczyzny:

```
[	24, -71 ],
[ 17, 23
],[ -22
, -68],[	-5
,
-4],
[	-100
, -50	], [	-25,	67
],[ 10,	43
], [ 65,	-68	],	[
-41	,	-2
],	[
34,
35 ]
```
Tekst ten naprawimy jednym użyciem `re.sub()`.

In [53]:
punkty = '''[	24, -71 ],
[ 17, 23
],[ -22
, -68],[	-5
,
-4],
[	-100
, -50	], [	-25,	67
],[ 10,	43
], [ 65,	-68	],	[
-41	,	-2
],	[
34,
35 ]'''

In [56]:
# Usuwamy wszystkie białe znaki
punkty = re.sub(r'\s+', '', punkty)

# Dokładamy spację po przecinku rozdzielającym punkty.
# W tym przypadku wystarczy metoda łańcuchów replace()
punkty.replace('],[', '], [')

'[24,-71], [17,23], [-22,-68], [-5,-4], [-100,-50], [-25,67], [10,43], [65,-68], [-41,-2], [34,35]'

## Literatura

Do poczytania:

* [HOWTO autorstwa A.M. Kuchlinga](https://docs.python.org/3.6/howto/regex.html)
* [Dive into Python, chapter 5](http://www.diveintopython3.net/regular-expressions.html)

Do obejrzenia:

* [Wykład A. Sweigarta](https://www.youtube.com/watch?v=abrcJ9MpF60&t=970s)
* [Wykład T. Hunnera](https://www.youtube.com/watch?v=0sOfhhduqks&t=1172s)